# GPU 환경 설정 가이드
이 노트북은 `gemini_gpu` conda 환경을 생성하고, LightGBM/XGBoost/cuML을 GPU 모드로 사용할 수 있도록 필요한 패키지를 설치/검증하는 절차를 정리합니다.

## 1. GPU 및 CUDA 드라이버 확인
GPU가 바르게 인식되는지 반드시 확인하세요. 해당 명령은 시스템에 설치된 NVIDIA 드라이버와 CUDA 버전을 출력합니다.

In [ ]:
!nvidia-smi

## 2. 새 conda 환경 생성
CUDA 11.8과 호환되는 Python 3.11 기반 환경을 생성합니다. 이미 존재한다면 이 단계는 건너뛰어도 됩니다.

In [ ]:
!conda create -n gemini_gpu python=3.11 pip -y

환경 활성화는 노트북이 아닌 터미널에서 실행합니다.
```bash
conda activate gemini_gpu
```

## 3. 기본 패키지 및 CUDA Toolkit 설치
데이터 처리 패키지와 `cudatoolkit 11.8`을 conda-forge 채널에서 받습니다.

In [ ]:
!conda install -n gemini_gpu -c conda-forge numpy pandas scikit-learn joblib cudatoolkit=11.8 -y

## 4. RAPIDS cuML 설치
ExtraTrees의 GPU 버전을 사용하려면 RAPIDS 스택이 필요합니다. 다운로드 용량이 크므로 안정적인 네트워크 환경에서 실행하세요. `mamba`를 사용하면 속도를 높일 수 있습니다.

In [ ]:
!conda install -n gemini_gpu -c rapidsai -c nvidia -c conda-forge cuml=24.12 python=3.11 -y

## 5. LightGBM & XGBoost GPU 빌드 설치
아래 명령은 PyPI wheel을 설치하지만, GPU 지원을 위해서는 CMake 빌드 옵션을 추가해야 할 수 있습니다. conda-forge의 GPU 지원 빌드를 사용하거나, 직접 빌드 시 `--config-settings=cmake.define.USE_GPU=ON` 등을 지정하세요. 임시로 CPU 빌드가 설치되었더라도 이후 재설치로 대체하면 됩니다.

In [ ]:
!conda run -n gemini_gpu pip install lightgbm==4.6.0 xgboost==3.1.2

GPU 빌드 예시 (필요 시 터미널에서 실행):
```bash
conda activate gemini_gpu
pip uninstall -y lightgbm
pip install --no-binary=lightgbm lightgbm --config-settings=cmake.define.USE_GPU=ON
```

## 6. 설치 검증
각 패키지가 GPU를 인식하는지 확인합니다. 오류가 발생하면 CUDA 경로 설정과 드라이버 버전을 다시 확인하세요.

In [ ]:
%%bash
conda run -n gemini_gpu python - <<'PY'
import lightgbm as lgb
import xgboost as xgb
from cuml.ensemble import ExtraTreesClassifier
print('LightGBM version:', lgb.__version__)
print('XGBoost version:', xgb.__version__)
print('cuML ExtraTrees OK')
PY

## 7. 학습 스크립트 실행
환경이 준비되면 아래와 같이 트리 학습 스크립트를 GPU 모드로 실행할 수 있습니다.

In [ ]:
!conda run -n gemini_gpu python 01_src/01_training/01_train_tree.py --model lightgbm --device gpu

필요 시 `xgboost`, `extratrees` 모델도 `--device gpu`로 실행해 성능을 비교하세요.